# Imports and paths configuration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

c:\Users\alois\Desktop\Advanced ML\AML_Task_1


In [3]:
import sys
import os
import yaml
from ax.service.ax_client import AxClient
from task1.src.train import main
from task1.utils import unflatten_dict

In [4]:
with open('task1/config/config_ada.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Parameters choice

In [5]:
parameters = [
    {
        "name": "mice.estimator_params.n_estimators",
        "type": "range",
        "bounds": [50, 200],
        "value_type": 'int'
    },
    {
        "name": "mice.estimator_params.max_depth",
        "type": 'range',
        'bounds': [2, 10],
        'value_type': 'int'
    },
    {
        'name': 'mice.mice_params.max_iter',
        'type': 'range',
        'bounds': [5, 20],
        'value_type': 'int'
    },
    {
        'name': 'mice.num_corr_features',
        'type': 'range',
        'bounds': [5, 150],
        'value_type': 'int'
    },
    {
        "name": "lasso_alpha", 
        "type": "range", 
        "bounds": [0, 10],
        "value_type": 'int'
    },
    {
        "name": "mice.simple_params.strategy",
        "type": "choice",
        "values": ["mean", "median", "most_frequent", "constant"],
    },
    {
        "name": "mice.mice_params.imputation_order",
        "type": "choice",
        "values": ["ascending", "descending", "roman", "arabic", "random"],
    },
    {
        "name": "n_estimators",
        "type": "range",
        "bounds": [10, 100],
        "value_type": "int"
    },
    {
        "name": "learning_rate",
        "type": "range",
        "bounds": [0.01, 1.0],
        "value_type": "float"
    }
]


# Tuning

In [6]:
def evaluate_model(parameters):
    # Change any default config here:
    config['train']['model_type'] = 'ada'
    
    config['train']['preprocessing']['feature_selection']['arguments']['alpha'] = parameters['lasso_alpha']/10
    config['train']['model_config']['n_estimators'] = parameters['n_estimators']
    config['train']['model_config']['learning_rate'] = parameters['learning_rate']

    config['train']['preprocessing']['imputation']['method'] = 'mice'
    config['train']['preprocessing']['imputation']['arguments'] = {}
    config['train']['preprocessing']['imputation']['arguments']['estimator_params'] = {}
    config['train']['preprocessing']['imputation']['arguments']['mice_params'] = {}
    config['train']['preprocessing']['imputation']['arguments']['simple_params'] = {}
    config['train']['preprocessing']['imputation']['arguments']['num_corr_features'] = parameters['mice.num_corr_features']


    
    # Parse parameters
    for param_name in parameters:
        param_value = parameters[param_name]
        if param_name.startswith('mice.'):
            param_name_split = param_name.split('.')
            if len(param_name_split) == 3:
                config['train']['preprocessing']['imputation']['arguments'][param_name_split[1]][param_name_split[2]] = param_value
            elif len(param_name_split) == 4:
                config['train']['preprocessing']['imputation']['arguments'][param_name_split[1]][param_name_split[2]][param_name_split[3]] = param_value
    
    print(config)
    return main(config)

In [7]:
ax_client = AxClient()
objective_name = 'preprocessing_tune'
ax_client.create_experiment(
    parameters=parameters,
    objective_name=objective_name,
    minimize=False
)

# Specify the initial parameters according to your configuration
initial_parameters = {
    "mice.estimator_params.n_estimators": 100,  # You can set the initial values as needed
    "mice.estimator_params.max_depth": 5,
    "mice.mice_params.max_iter": 10,
    "mice.num_corr_features": 50,
    "lasso_alpha": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "mice.simple_params.strategy": "mean",  # Choose a strategy from your configuration
    "mice.mice_params.imputation_order": "descending"
}

_, trial_id = ax_client.attach_trial(initial_parameters)
ax_client.complete_trial(trial_id, raw_data=evaluate_model(initial_parameters))






for i in range(50):    
    curr_parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate_model(curr_parameters))

[INFO 11-12 13:05:09] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 11-12 13:05:09] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter mice.simple_params.strategy. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
c:\Users\alois\anaconda3\Lib\site-packages\ax\core\parameter.py:517: UserWarning: `is_ordered` is not specified for `ChoiceParameter` "mice.simple_params.strategy". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.
  warn(
c:\Users\alois\anaconda3\Lib\site-packages\ax\core\parameter.py:517: UserWarning: `sort_values` is not specified for `ChoiceParameter` "mice.simple_params.s

{'train': {'data_paths': {'features': 'data/X_train.csv', 'label': 'data/y_train.csv'}, 'preprocessing': {'scale': True, 'imputation': {'method': 'mice', 'arguments': {'estimator_params': {'n_estimators': 100, 'max_depth': 5}, 'mice_params': {'max_iter': 10, 'imputation_order': 'descending'}, 'simple_params': {'strategy': 'mean'}, 'num_corr_features': 50}}, 'outliers_detection': {'method': 'isolation_forest', 'arguments': {}}, 'feature_selection': {'method': 'lasso', 'arguments': {'alpha': 0.5}}, 'execution_order': ['imputation', 'outliers_detection', 'feature_selection']}, 'model_type': 'ada', 'model_config': {'n_estimators': 50, 'learning_rate': 0.1}, 'cross_validation': 5, 'results_path': 'results/overview.csv'}, 'evaluate': {'run': True, 'data_paths': {'features': 'data/X_test.csv', 'label': 'data/y_test_ada_mice.csv'}}}


c:\Users\alois\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Number of selected features: 39


c:\Users\alois\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

c:\Users\alois\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Number of selected features: 27


[INFO 11-12 16:56:45] ax.service.ax_client: Completed trial 17 with data: {'preprocessing_tune': (0.416992, None)}.
[INFO 11-12 16:56:45] ax.service.ax_client: Generated new trial 18 with parameters {'mice.estimator_params.n_estimators': 74, 'mice.estimator_params.max_depth': 7, 'mice.mice_params.max_iter': 9, 'mice.num_corr_features': 45, 'lasso_alpha': 0, 'n_estimators': 48, 'learning_rate': 0.267642, 'mice.simple_params.strategy': 'mean', 'mice.mice_params.imputation_order': 'roman'}.


{'train': {'data_paths': {'features': 'data/X_train.csv', 'label': 'data/y_train.csv'}, 'preprocessing': {'scale': True, 'imputation': {'method': 'mice', 'arguments': {'estimator_params': {'n_estimators': 74, 'max_depth': 7}, 'mice_params': {'max_iter': 9, 'imputation_order': 'roman'}, 'simple_params': {'strategy': 'mean'}, 'num_corr_features': 45}}, 'outliers_detection': {'method': 'isolation_forest', 'arguments': {}}, 'feature_selection': {'method': 'lasso', 'arguments': {'alpha': 0.0}}, 'execution_order': ['imputation', 'outliers_detection', 'feature_selection']}, 'model_type': 'ada', 'model_config': {'n_estimators': 48, 'learning_rate': 0.26764179673045874}, 'cross_validation': 5, 'results_path': 'results/overview.csv'}, 'evaluate': {'run': True, 'data_paths': {'features': 'data/X_test.csv', 'label': 'data/y_test_ada_mice.csv'}}}


KeyboardInterrupt: 

# Get best config and write prediction data

In [8]:
import pandas as pd
results_path = 'results/overview.csv'
# Remove duplicates results, should fix this in train.py
pd.read_csv(results_path).sort_values(by='train_results', ascending=False).drop_duplicates().to_csv(results_path, index=False)
results = pd.read_csv(results_path).sort_values(by='train_results', ascending=False).drop_duplicates().reset_index(drop=True)

In [9]:
results

,train/data_paths/features,train/data_paths/label,train/preprocessing/scale,train/preprocessing/imputation/method,train/preprocessing/imputation/arguments/n_neighbors,train/preprocessing/outliers_detection/method,train/preprocessing/feature_selection/method,train/preprocessing/feature_selection/arguments/alpha,train/preprocessing/execution_order,train/model_type,...,evaluate/run,evaluate/data_paths/features,evaluate/data_paths/label,train_results,train/preprocessing/imputation/arguments/estimator_params/n_estimators,train/preprocessing/imputation/arguments/estimator_params/max_depth,train/preprocessing/imputation/arguments/mice_params/max_iter,train/preprocessing/imputation/arguments/mice_params/imputation_order,train/preprocessing/imputation/arguments/simple_params/strategy,train/preprocessing/imputation/arguments/num_corr_features
0,data/X_train.csv,data/y_train.csv,True,knn,20.0,isolation_forest,lasso,0.7,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.448614,NaN,NaN,NaN,NaN,NaN,NaN
1,data/X_train.csv,data/y_train.csv,True,knn,20.0,isolation_forest,lasso,0.7,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.445853,NaN,NaN,NaN,NaN,NaN,NaN
2,data/X_train.csv,data/y_train.csv,True,knn,20.0,isolation_forest,lasso,0.8,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.445686,NaN,NaN,NaN,NaN,NaN,NaN
3,data/X_train.csv,data/y_train.csv,True,knn,8.0,isolation_forest,lasso,1.0,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.443252,NaN,NaN,NaN,NaN,NaN,NaN
4,data/X_train.csv,data/y_train.csv,True,knn,19.0,isolation_forest,lasso,0.8,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.441705,NaN,NaN,NaN,NaN,NaN,NaN
5,data/X_train.csv,data/y_train.csv,True,knn,18.0,isolation_forest,lasso,0.9,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.439079,NaN,NaN,NaN,NaN,NaN,NaN
6,data/X_train.csv,data/y_train.csv,True,knn,20.0,isolation_forest,lasso,0.6,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.428652,NaN,NaN,NaN,NaN,NaN,NaN
7,data/X_train.csv,data/y_train.csv,True,knn,15.0,isolation_forest,lasso,0.8,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.428306,NaN,NaN,NaN,NaN,NaN,NaN
8,data/X_train.csv,data/y_train.csv,True,knn,13.0,isolation_forest,lasso,0.6,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.423359,NaN,NaN,NaN,NaN,NaN,NaN
9,data/X_train.csv,data/y_train.csv,True,knn,8.0,isolation_forest,lasso,0.4,imputation; outliers_detection; feature_selection,ada,...,True,data/X_test.csv,data/y_test_ada.csv,0.422158,NaN,NaN,NaN,NaN,NaN,NaN


#### Get best params and rerun the trial to get predictions

In [10]:
def get_best_params(ax_client):
    print(ax_client.experiment.fetch_data().df.sort_values(by='mean', ascending=False).iloc[0])
    best_trial_idx = ax_client.experiment.fetch_data().df.sort_values(by='mean', ascending=False).iloc[0].trial_index
    best_params = ax_client.experiment.trials[best_trial_idx].arm.parameters
    return best_params


In [14]:
initial_params = get_best_params(ax_client)

arm_name                      6_0
metric_name    preprocessing_tune
mean                     0.421865
sem                           NaN
trial_index                     6
Name: 6, dtype: object


In [15]:
# Rerun trial nb_trial
ax_client = AxClient()
objective_name = 'preprocessing_tune'
ax_client.create_experiment(
    parameters=parameters,
    objective_name=objective_name,
    minimize=False
)
_, trial_id = ax_client.attach_trial(initial_params)
ax_client.complete_trial(trial_id, raw_data=evaluate_model(initial_params))

for i in range(10):    
    curr_parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate_model(curr_parameters))

[INFO 11-12 16:58:39] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 11-12 16:58:39] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter mice.simple_params.strategy. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
c:\Users\alois\anaconda3\Lib\site-packages\ax\core\parameter.py:517: UserWarning: `is_ordered` is not specified for `ChoiceParameter` "mice.simple_params.strategy". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.
  warn(
c:\Users\alois\anaconda3\Lib\site-packages\ax\core\parameter.py:517: UserWarning: `sort_values` is not specified for `ChoiceParameter` "mice.simple_params.s

{'train': {'data_paths': {'features': 'data/X_train.csv', 'label': 'data/y_train.csv'}, 'preprocessing': {'scale': True, 'imputation': {'method': 'mice', 'arguments': {'estimator_params': {'n_estimators': 126, 'max_depth': 7}, 'mice_params': {'max_iter': 19, 'imputation_order': 'descending'}, 'simple_params': {'strategy': 'mean'}, 'num_corr_features': 86}}, 'outliers_detection': {'method': 'isolation_forest', 'arguments': {}}, 'feature_selection': {'method': 'lasso', 'arguments': {'alpha': 0.7}}, 'execution_order': ['imputation', 'outliers_detection', 'feature_selection']}, 'model_type': 'ada', 'model_config': {'n_estimators': 62, 'learning_rate': 0.6395189069211483}, 'cross_validation': 5, 'results_path': 'results/overview.csv'}, 'evaluate': {'run': True, 'data_paths': {'features': 'data/X_test.csv', 'label': 'data/y_test_ada_mice.csv'}}}


c:\Users\alois\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Number of selected features: 24


c:\Users\alois\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


...

Number of selected features: 20


c:\Users\alois\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Number of selected features: 23


[INFO 11-12 17:37:43] ax.service.ax_client: Completed trial 0 with data: {'preprocessing_tune': (0.405412, None)}.


NameError: name 'n2' is not defined

In [ ]:
best_param_dct_linear = results[results['train/model_type'] == 'ada'].iloc[0].to_dict()
del best_param_dct_linear['train_results']

In [ ]:
best_param_dct_linear

In [ ]:
best_param_config = unflatten_dict(best_param_dct_linear)
best_param_config['evaluate']['data_paths']['label'] = 'results/y_test_svm.csv'
best_param_config['evaluate']['run'] = True
best_param_config['train']['model_config'] = {}
best_param_config['train']['preprocessing']['outliers_detection']['arguments'] = {}

In [ ]:
main(best_param_config)

In [ ]:
pd.read_csv('results/overview.csv').sort_values(by='train_results', ascending=False).drop_duplicates().reset_index(drop=True)